# 🧙‍♂️ DisSysLab — Colab Wizard (Track A)

Welcome! This notebook guides non-programmers to build their **first distributed app** with DisSysLab.

**How it works:** We’ll take *small steps*. You’ll see a diagram of the distributed application network that you have created, the results of executing the application, and the generated Python code.

> If you are a programmer, use Track C (lessons in README files) in the repo.


## ✅ Colab Tips
- Run cells from top to bottom (Shift+Enter).
- Fields marked with ▶️ **FORM** can be edited without touching code.
- You can reset a choice at any time and re-run the **Build & Run** cell.


## 🛠 Install DisSysLab
DisSysLab is currently **private**, so installation options:
1. **Clone with a Personal Access Token (PAT)** *(recommended for maintainers)*, or
2. **Upload a wheel/zip** to this Colab (drag into the left Files pane), then `pip install` it.

When the repo is public, this cell will be a simple `pip install dissyslab[examples]`.


In [ ]:
#@title (Maintainer) Install from GitHub using a token *(optional for now)*
use_token = False #@param {type:"boolean"}
github_user = "kmchandy" #@param {type:"string"}
repo_name = "DisSysLab" #@param {type:"string"}
branch = "main" #@param {type:"string"}
token = "" #@param {type:"string"}

if use_token and token:
    print("Installing DisSysLab from private GitHub repo…")
    get_ipython().system('pip -q install git+https://{token}@github.com/{github_user}/{repo_name}.git@{branch}')
else:
    print("Skipping install for now (repo is private). We'll simulate the run.")


In [ ]:
#@title (Optional) Mount Google Drive to save apps and outputs
mount_drive = False #@param {type:"boolean"}
if mount_drive:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Drive mounted. We'll save files to /content/drive/MyDrive/DisSysLab/")
else:
    print("Not mounting Drive. Files will stay in this Colab session.")


In [ ]:
#@title ▶️ Scene 1 — Choose a template
template = "Text analysis" #@param ["Text analysis", "Number crunching", "Custom"]
print(f"Template selected: {template}")


In [ ]:
#@title ▶️ Scene 2 — Where is your data?
source_kind = "Type it in" #@param ["Type it in", "Upload a file", "URL or API"]
sample_text = """I love sunny days\nI hate traffic jams\nThis pizza is amazing""" #@param {type:"string"}
print(f"Source: {source_kind}")


In [ ]:
#@title ▶️ Scene 3 — What should we do with the data?
if template == "Text analysis":
    processing = "Sentiment analysis" #@param ["Sentiment analysis", "Summarize", "Extract entities", "Custom prompt"]
else:
    processing = "Scale numbers" #@param ["Scale numbers", "Average", "Apply formula", "Custom function"]
print(f"Processing: {processing}")


In [ ]:
#@title ▶️ Scene 4 — Where should results go?
show_on_screen = True #@param {type:"boolean"}
save_to_file = False #@param {type:"boolean"}
print(f"Show on screen: {show_on_screen}, Save to file: {save_to_file}")


## 🔍 Review
We are about to build this network:

```
[ Generator ] → [ Transformer ] → [ Recorder ]
```

If "Upload a file" was selected, you'll be prompted in the next cell.


In [ ]:
import json, os
print("⚡ Building your app…")

texts = []
if source_kind == "Type it in":
    texts = [t.strip() for t in sample_text.splitlines() if t.strip()]
elif source_kind == "Upload a file":
    print("Please upload a text file in the left Files pane, then set its path below and re-run.")
    upload_path = "/content/input.txt"  # change if needed
    if os.path.exists(upload_path):
        with open(upload_path, 'r') as f:
            texts = [t.strip() for t in f if t.strip()]
    else:
        print("Oops — couldn't find /content/input.txt. Using sample text.")
        texts = [t.strip() for t in sample_text.splitlines() if t.strip()]
else:
    print("URL/API mode not implemented in this skeleton. Using sample text.")
    texts = [t.strip() for t in sample_text.splitlines() if t.strip()]

print(f"Loaded {len(texts)} lines.")

try:
    from dsl.core import Network
    from dsl.block_lib.stream_generators import generate
    from dsl.block_lib.stream_transformers import SentimentClassifierWithGPT
    from dsl.block_lib.stream_recorders import RecordToList
    HAVE_DSL = True
except Exception:
    HAVE_DSL = False
    print("Note: DisSysLab not installed — simulating the pipeline.")

results = []

if HAVE_DSL and template == "Text analysis" and processing == "Sentiment analysis":
    net = Network(
        blocks={
            "gen": generate(texts, key="text"),
            "sentiment": SentimentClassifierWithGPT(input_key="text", output_key="sentiment"),
            "rec": RecordToList(results),
        },
        connections=[
            ("gen", "out", "sentiment", "in"),
            ("sentiment", "out", "rec", "in"),
        ],
    )
    net.compile_and_run()
else:
    def fake_sentiment(s):
        s_low = s.lower()
        if any(k in s_low for k in ["love", "amazing", "great", "good", "happy"]):
            return "positive"
        if any(k in s_low for k in ["hate", "bad", "angry", "terrible"]):
            return "negative"
        return "neutral"
    for t in texts:
        results.append({"text": t, "sentiment": fake_sentiment(t)})

print("\n✅ Results:")
for r in results:
    print(r)

if save_to_file:
    out_path = "/content/results.json"
    with open(out_path, 'w') as f:
        json.dump(results, f, indent=2)
    print(f"\nSaved to {out_path}")


## 🖼 Network Diagram
```
[ Generator ] → [ Transformer ] → [ Recorder ]
```
![Pipeline](/docs/images/simple_network.svg)


## 🎓 What you just did
1. **Created blocks** — chose type, gave a name, specified what it does.
2. **Connected blocks** — output from one flows to the next.

This is the foundation. You can now add more steps or switch data sources.
